<a href="https://colab.research.google.com/github/tsanzxc456/NLP/blob/master/lab2_0760054.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1

## Fetching the Corpus

In [1]:
import string
import pandas as pd
!pip install Unidecode
from unidecode import unidecode

     |████████████████████████████████| 245kB 4.7MB/s 


In [2]:
def get_corpus():
  df = pd.read_csv('https://raw.githubusercontent.com/bshmueli/108-nlp/master/reuters.csv') # https://bit.ly/nlp-reuters
  corpus = df.content.to_list()
  for i in range(0,len(corpus)):
    corpus[i] = unidecode(corpus[i])
  return corpus

## NLTK tokenizers and POS tagging

In [3]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')
import time


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
def doc_tokenizer(document):
  token = nltk.word_tokenize(document)
  lemmatizer = WordNetLemmatizer()
  word_list = []
  token_pos = nltk.pos_tag(token)
  for i in range(0,len(token_pos)):
    if token_pos[i][1] == 'NNP':
      word_list.append(token[i])
  return word_list

In [5]:
def get_vocab(corpus):
  vocab_list = []
  for i in range(0,len(corpus)):
    tokens = doc_tokenizer(corpus[i])
    vocab_list.extend(tokens)
  return vocab_list

## Computing bigrams

In [6]:
from collections import Counter
from nltk import ngrams

In [7]:
def get_most_5_bigrams(corpus):
  bigrams_counter = Counter()
  bigrams_counter.update(ngrams(get_vocab(corpus),2))
  most_5_bigrams = bigrams_counter.most_common(5)
  for i in range(0,len(most_5_bigrams)):
    print('*{} >> ({})'.format(most_5_bigrams[i][0], most_5_bigrams[i][1]))



## Testing !!!

In [8]:
corpus = get_corpus()
get_most_5_bigrams(corpus)

*('U.', 'S.') >> (17601)
*('Donald', 'Trump') >> (3222)
*('New', 'York') >> (2443)
*('Islamic', 'State') >> (1927)
*('President', 'Donald') >> (1924)


# Assignment 2

## Fetching the Corpus

In [9]:
!pip install Unidecode
import pandas as pd
import string
import spacy
import time
nlp = spacy.load("en_core_web_sm")
from unidecode import unidecode

In [10]:
def get_corpus():
  df = pd.read_csv('https://raw.githubusercontent.com/bshmueli/108-nlp/master/buzzfeed.csv')
  corpus = df.content.to_list()
  for i in range(0,len(corpus)):
    if type(corpus[i]) == type(0.1):
      corpus[i] = "hi"
    else:
      corpus[i] = unidecode(corpus[i])
  return corpus

In [11]:
def get_corpus_title():
  df = pd.read_csv('https://raw.githubusercontent.com/bshmueli/108-nlp/master/buzzfeed.csv')
  corpus_title = df.title.to_list()
  return corpus_title

## Spacy tokenizers and POS tagging

In [12]:
def tokenize(document):
  doc = nlp(document)
  tokens = [token for token in doc if not token.is_stop]
  tokens = [token for token in tokens if not token.is_punct]
  tokens = [token for token in tokens if not token.is_space]
  for i in range(0,len(tokens)):
    tokens[i] = tokens[i].lemma_ + '_' + tokens[i].pos_
  return tokens


In [13]:
from collections import Counter

In [14]:
def get_vocab_from_corpus(corpus):
  vocabulary = Counter()
  for i in range(0,len(corpus)):
    tokens = tokenize(corpus[i])
    vocabulary.update(tokens)
  return vocabulary

In [15]:
def get_vocab_from_document(document):
  vocabulary = Counter()
  tokens = tokenize(document)
  vocabulary.update(tokens)
  return vocabulary

## Computing TFIDF score

In [16]:
import math

In [17]:
def get_doc_vocab_list(corpus):
    doc_vocab_list = []
    for i in range(0,len(corpus)):
      tmp_doc_vocab = get_vocab_from_document(corpus[i])
      doc_vocab_list.append(tmp_doc_vocab)
    return doc_vocab_list


In [18]:
def get_df(word_list):
  df_list = []
  for word in word_list:
    tmp_df = 0
    for i in range(0,len(corpus)):
      if doc_vocab_list[i][word[0]]!=0:
        tmp_df = tmp_df + 1
    df_list.append(tmp_df)
  return df_list

In [19]:
def tfidf_score(num,key_word,doc_id):
  tf = doc_vocab_list[doc_id][key_word]/len(list(doc_vocab_list[doc_id].elements()))
  idf = math.log10(len(corpus)/df_list[num])
  tfidf = tf * idf
  return tfidf

In [20]:
def tfidf_vector(doc_id):
  tfidf_vec = []
  for i in range(0,len(top_512_vocab)):   
    tfidf_vec.append(tfidf_score(i,top_512_vocab[i][0],doc_id))
  return tfidf_vec

In [21]:
def cosine_similarity(vec_a, vec_b):
  assert len(vec_a) == len(vec_b)
  if sum(vec_a) == 0 or sum(vec_b) == 0:
    return 0 # hack
  a_b = sum(i[0] * i[1] for i in zip(vec_a, vec_b))
  a_2 = sum([i*i for i in vec_a])
  b_2 = sum([i*i for i in vec_b])
  return a_b/(math.sqrt(a_2) * math.sqrt(b_2))

In [22]:
def doc_similarity(seed_vector, doc_id):
  final_score = cosine_similarity(seed_vector, tfidf_vector(doc_id))
  return final_score

In [23]:
def k_similar(seed_id, k):
  seed_doc = corpus[seed_id]
  seed_title = corpus_title[seed_id]
  seed_tfidf_vector = tfidf_vector(seed_id)
  print('> "{}"'.format(seed_title))
  similarities = [doc_similarity(seed_tfidf_vector, doc_id) for id, doc_id in enumerate(range(0,len(corpus)))]
  top_indices = sorted(range(len(similarities)), key=lambda i: similarities[i])[-k:] # https://stackoverflow.com/questions/13070461/get-indices-of-the-top-n-values-of-a-list
  nearest = [[corpus_title[id], similarities[id]] for id in top_indices]
  print()
  for story in reversed(nearest):
    print('* "{}" ({})'.format(story[0], story[1]))

## Testing !!!

In [24]:
corpus = get_corpus()
corpus_title = get_corpus_title()
top_512_vocab = get_vocab_from_corpus(corpus).most_common(512)
doc_vocab_list = get_doc_vocab_list(corpus)
df_list = get_df(top_512_vocab)
k_similar(54,5)

> "Clinton: No, I Do Have Young Enthusiastic Supporters"

* "Clinton: No, I Do Have Young Enthusiastic Supporters" (1.0)
* "Clinton Campaign Targets LGBT Voters In Push For Decisive New York Win" (0.6138809125908788)
* "Why Bill Clinton’s Voters Wouldn’t Vote For Him Today, As Told By Bill Clinton" (0.5496478797654698)
* "New Clinton Spanish-Language Ad Highlights “First-Time” Voters Opposing Trump" (0.540856788882674)
* "Turning To The General, Clinton Makes Pitch To The “Thoughtful Republican”" (0.5276332843231093)
